# MARATONA BEHIND THE CODE 2020

## DESAFIO 2: PARTE 2

In [2]:
!pip install scikit-learn==0.19

     |████████████████████████████████| 12.4MB 8.5MB/s eta 0:00:01
ERROR: category-encoders 2.0.0 has requirement scikit-learn>=0.20.0, but you'll have scikit-learn 0.19.0 which is incompatible.
ERROR: autoai-libs 1.10.5 has requirement numpy>=1.16.4, but you'll have numpy 1.15.4 which is incompatible.
ERROR: autoai-libs 1.10.5 has requirement pandas>=0.24.2, but you'll have pandas 0.24.1 which is incompatible.
ERROR: autoai-libs 1.10.5 has requirement scikit-learn==0.20.3, but you'll have scikit-learn 0.19.0 which is incompatible.
  Found existing installation: scikit-learn 0.20.3
    Uninstalling scikit-learn-0.20.3:
      Successfully uninstalled scikit-learn-0.20.3


In [1]:
import json
import requests
import pandas as pd
import numpy as np
import sklearn 
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_validate

from xgboost import XGBClassifier


0.19.0


In [2]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_2e29cc98c92249589fb271f5cf851e63 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Z3iPsIkTFB1QX-JXUvJPVxnb1_OYJmsaClKhR0Inva2x',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_2e29cc98c92249589fb271f5cf851e63.get_object(Bucket='desafio2-donotdelete-pr-a4rvittqna2hv9',Key='dataset_desafio_2.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,MATRICULA,NOME,REPROVACOES_DE,REPROVACOES_EM,REPROVACOES_MF,REPROVACOES_GO,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,INGLES,H_AULA_PRES,TAREFAS_ONLINE,FALTAS,PERFIL
0,502375,Márcia Illiglener,0,0,0,0,6.2,5.8,4.6,5.9,0.0,2,4,3,EXATAS
1,397093,Jason Jytereoman Izoimum,0,0,0,0,6.0,6.2,5.2,4.5,1.0,2,4,3,EXATAS
2,915288,Bartolomeu Inácio da Gama,0,0,0,0,7.3,6.7,7.1,7.2,0.0,5,0,3,HUMANAS
3,192652,Fernanda Guedes,1,3,1,1,0.0,0.0,0.0,0.0,1.0,4,4,4,DIFICULDADE
4,949491,Alessandre Borba Gomes,1,3,1,1,0.0,0.0,0.0,0.0,1.0,5,2,5,DIFICULDADE


### Construção da Pipeline completa para encapsulamento no WML

#### Preparando transformações personalizadas para carregamento no WML

In [12]:
# substitua o link abaixo pelo link do seu repositório git (se for o caso)
!git clone https://github.com/uedermdscardoso/sklearn_transforms.git

Cloning into 'sklearn_transforms'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 21 (delta 4), reused 18 (delta 1), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [3]:
!cd sklearn_transforms
!ls -ltr

total 4
drwxr-x--- 5 dsxuser dsxuser 4096 Aug 20 16:41 sklearn_transforms


Para subir o código no WML, precisamos enviar um arquivo .zip com todo o código fonte, então iremos zipar o diretório clonado em seguida:

In [4]:
!zip -r sklearn_transforms.zip sklearn_transforms

  adding: sklearn_transforms/ (stored 0%)
  adding: sklearn_transforms/setup.cfg (deflated 16%)
  adding: sklearn_transforms/setup.py (deflated 46%)
  adding: sklearn_transforms/my_custom_sklearn_transforms.egg-info/ (stored 0%)
  adding: sklearn_transforms/my_custom_sklearn_transforms.egg-info/SOURCES.txt (deflated 48%)
  adding: sklearn_transforms/my_custom_sklearn_transforms.egg-info/not-zip-safe (stored 0%)
  adding: sklearn_transforms/my_custom_sklearn_transforms.egg-info/dependency_links.txt (stored 0%)
  adding: sklearn_transforms/my_custom_sklearn_transforms.egg-info/top_level.txt (stored 0%)
  adding: sklearn_transforms/my_custom_sklearn_transforms.egg-info/PKG-INFO (deflated 33%)
  adding: sklearn_transforms/my_custom_sklearn_transforms/ (stored 0%)
  adding: sklearn_transforms/my_custom_sklearn_transforms/sklearn_transformers.py (deflated 60%)
  adding: sklearn_transforms/my_custom_sklearn_transforms/__init__.py (stored 0%)
  adding: sklearn_transforms/.git/ (stored 0%)
  ad

Com o arquivo zip do nosso pacote carregado no Kernel deste notebook, podemos utilizar a ferramenta pip para instalá-lo, conforme a célula abaixo:

In [5]:
!pip install sklearn_transforms.zip

Processing ./sklearn_transforms.zip
  Stored in directory: /home/dsxuser/.tmp/pip-ephem-wheel-cache-q47e0dwk/wheels/8f/88/32/f886e7510a37b111e2a1b7e689e04450acda46732970a7ed78
Successfully built my-custom-sklearn-transforms
  Found existing installation: my-custom-sklearn-transforms 1.0
    Uninstalling my-custom-sklearn-transforms-1.0:
      Successfully uninstalled my-custom-sklearn-transforms-1.0


Podemos agora realizar a importação do nosso pacote personalizado em nosso notabook!

Iremos importar a transformação DropColumns. Se você possui outras transformações personalizadas, não se esqueça de importá-las!

In [6]:
from my_custom_sklearn_transforms.sklearn_transformers import DropColumns,FillNaNColumns

In [8]:
# Criação de uma Transform personalizada ``DropColumns``

fill_nan_columns = FillNaNColumns(
    columns=["MATRICULA","NOME","H_AULA_PRES","TAREFAS_ONLINE","FALTAS","NOTA_DE","NOTA_EM","NOTA_MF","NOTA_GO","INGLES","REPROVACOES_DE","REPROVACOES_EM","REPROVACOES_MF","REPROVACOES_GO"]
)

rm_columns = DropColumns(
    columns=["MATRICULA","NOME","H_AULA_PRES","TAREFAS_ONLINE","FALTAS","REPROVACOES_DE","REPROVACOES_EM","REPROVACOES_MF","REPROVACOES_GO"]
)

In [9]:
# Definição das colunas que serão features (nota-se que a coluna NOME não está presente)
features = [
    "MATRICULA", "NOME", 'REPROVACOES_DE', 'REPROVACOES_EM', "REPROVACOES_MF", "REPROVACOES_GO",
    "NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO",
    "INGLES", "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
]

# Definição da variável-alvo
target = ["PERFIL"]

# Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
X = df_data_1[features]
y = df_data_1[target]

In [10]:
# Separação dos dados em um conjunto de treino e um conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=337)

In [12]:
# Criação da nossa pipeline para armazenamento no Watson Machine Learning:
my_pipeline = Pipeline(
    steps=[
        ('fill_nan_columns', fill_nan_columns),
        ('remove_cols', rm_columns),
        ('dtc', XGBClassifier(
             base_score=0.5,
             booster='gbtree',
             colsample_bylevel=1,
             colsample_bynode=1,
             colsample_bytree=1,
             gamma=0,
             learning_rate=0.1,
             max_delta_step=0,
             max_depth=3,
             max_features='sqrt',
             min_child_weight=1,
             min_samples_split=2,
             missing=None,
             n_estimators=50,
             n_jobs=1,
             nthread=None,
             objective='multi:softprob',
             random_state=0,
             reg_alpha=0,
             reg_lambda=1,
             scale_pos_weight=1,
             seed=None,
             subsample=1,
        )),
    ]
)


In [13]:
# Inicialização da Pipeline (pré-processamento e realização do treinamento do modelo)
my_pipeline.fit(X_train, y_train.values.ravel())

Pipeline(memory=None,
     steps=[('fill_nan_columns', FillNaNColumns(columns=['MATRICULA', 'NOME', 'H_AULA_PRES', 'TAREFAS_ONLINE', 'FALTAS', 'NOTA_DE', 'NOTA_EM', 'NOTA_MF', 'NOTA_GO', 'INGLES', 'REPROVACOES_DE', 'REPROVACOES_EM', 'REPROVACOES_MF', 'REPROVACOES_GO'],
        fill=0)), ('remove_cols', DropColumns(columns=['M...tate=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))])

### Encapsulando uma Pipeline personalizada no Watson Machine Learning

#### Estabelecendo conexão entre o cliente Python do WML e a sua instância do serviço na nuvem

In [14]:
# Biblioteca Python com implementação de um cliente HTTP para a API do WML
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [15]:
wml_credentials = {
  "apikey": "WuKCuD1oo4b5izUwW5mIyZxErhC3Dm8MRwnjLk4ERhpg",
  "iam_apikey_description": "Auto-generated for key ae5ff84b-bd0e-48bd-b443-eb3b7a10d288",
  "iam_apikey_name": "Credenciais de serviço-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/4f1bc72e49e441cbab8e725a7faa5972::serviceid:ServiceId-ded09920-5f7b-4f49-bc91-c20772a4a48f",
  "instance_id": "e9155f37-76f9-4b5d-a011-7eaf4c0b10f6",
  "url": "https://us-south.ml.cloud.ibm.com"
}

In [16]:
# Instanciando um objeto cliente do Watson Machine Learning a partir das credenciais fornecidas

clientWML = WatsonMachineLearningAPIClient(wml_credentials)

In [17]:
# Extraindo detalhes da sua instância do Watson Machine Learning

instance_details = clientWML.service_instance.get_details()
print(json.dumps(instance_details, indent=4))

{
    "entity": {
        "source": "Bluemix",
        "published_models": {
            "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/e9155f37-76f9-4b5d-a011-7eaf4c0b10f6/published_models"
        },
        "usage": {
            "capacity_units": {
                "current": 0,
                "limit": 180000000
            },
            "computation_time": {
                "current": 0,
                "limit": 180000
            },
            "deployment_count": {
                "current": 0,
                "limit": 5
            },
            "expiration_date": "2020-09-01T00:00:00.000Z",
            "gpu_count_k80": {
                "current": 0,
                "limit": 8
            },
            "gpu_count_p100": {
                "current": 0,
                "limit": 0
            },
            "gpu_count_v100": {
                "current": 0,
                "limit": 0
            },
            "model_count": {
                "current": 1,
         

#### Listando todos os artefatos armazenados no seu WML

Para listar todos os artefatos armazenados em seu Watson Machine Learning, você pode usar a seguinte função:

    clientWML.repository.list()

In [21]:
# Listando todos os artefatos atualmente armazenados na sua instância do WML

clientWML.repository.list()

------------------------------------  -----------------------------  ------------------------  -----------------  --------------
GUID                                  NAME                           CREATED                   FRAMEWORK          TYPE
d17740e4-6f74-48ec-b4a8-192d56cb431e  desafio-2-mbtc2020-pipeline-1  2020-08-20T16:23:16.430Z  scikit-learn-0.20  model
2c13ace2-6793-40a5-88ac-6c3e058c711a  my_custom_sklearn_transform_1  2020-08-20T16:22:59.435Z  -                  python library
89f62cc9-17b3-4a5a-9c2f-68817be13f90  my_custom_wml_runtime_1        2020-08-20T16:23:06.279Z  -                  python runtime
------------------------------------  -----------------------------  ------------------------  -----------------  --------------


In [66]:
artifact_guid = "860c27b0-f5ce-4139-b7a4-3c1854ec54fb"
clientWML.repository.delete(artifact_guid)


'SUCCESS'

#### Criando uma nova definição de pacote Python personalizado no WML

In [22]:
# Definição de metadados do nosso pacote com as Transforms personalizadas
pkg_meta = {
    clientWML.runtimes.LibraryMetaNames.NAME: "my_custom_sklearn_transform_2",
    clientWML.runtimes.LibraryMetaNames.DESCRIPTION: "A custom sklearn transform",
    clientWML.runtimes.LibraryMetaNames.FILEPATH: "sklearn_transforms.zip",  # Note que estamos utilizando o .zip criado anteriormente!
    clientWML.runtimes.LibraryMetaNames.VERSION: "1.0",
    clientWML.runtimes.LibraryMetaNames.PLATFORM: { "name": "python", "versions": ["3.6"] }
}
custom_package_details = clientWML.runtimes.store_library( pkg_meta )
custom_package_uid = clientWML.runtimes.get_library_uid( custom_package_details )

print("\n Lista de artefatos de runtime armazenados no WML:")
clientWML.repository.list()


 Lista de artefatos de runtime armazenados no WML:
------------------------------------  -----------------------------  ------------------------  -----------------  --------------
GUID                                  NAME                           CREATED                   FRAMEWORK          TYPE
d17740e4-6f74-48ec-b4a8-192d56cb431e  desafio-2-mbtc2020-pipeline-1  2020-08-20T16:23:16.430Z  scikit-learn-0.20  model
dfde0b21-31fa-4258-8f8a-e06cf2528772  my_custom_sklearn_transform_2  2020-08-20T16:47:01.758Z  -                  python library
2c13ace2-6793-40a5-88ac-6c3e058c711a  my_custom_sklearn_transform_1  2020-08-20T16:22:59.435Z  -                  python library
89f62cc9-17b3-4a5a-9c2f-68817be13f90  my_custom_wml_runtime_1        2020-08-20T16:23:06.279Z  -                  python runtime
------------------------------------  -----------------------------  ------------------------  -----------------  --------------


In [23]:
runtime_meta = {
    clientWML.runtimes.ConfigurationMetaNames.NAME: "my_custom_wml_runtime_2",
    clientWML.runtimes.ConfigurationMetaNames.DESCRIPTION: "A Python runtime with custom sklearn Transforms",
    clientWML.runtimes.ConfigurationMetaNames.PLATFORM: {
        "name": "python",
        "version": "3.6"
    },
    clientWML.runtimes.ConfigurationMetaNames.LIBRARIES_UIDS: [ custom_package_uid ]
}
runtime_details = clientWML.runtimes.store( runtime_meta )
custom_runtime_uid = clientWML.runtimes.get_uid( runtime_details )

print("\n Detalhes do runtime armazenado:")
print(json.dumps(runtime_details, indent=4))


 Detalhes do runtime armazenado:
{
    "entity": {
        "name": "my_custom_wml_runtime_2",
        "description": "A Python runtime with custom sklearn Transforms",
        "custom_libraries": [
            {
                "name": "my_custom_sklearn_transform_2",
                "url": "https://private.us-south.ml.cloud.ibm.com/v4/libraries/dfde0b21-31fa-4258-8f8a-e06cf2528772",
                "version": "1.0"
            }
        ],
        "content_url": "https://private.us-south.ml.cloud.ibm.com/v4/runtimes/448df622-7b06-4fab-a92f-ad2a978f1a2a/content",
        "platform": {
            "name": "python",
            "version": "3.6"
        }
    },
    "metadata": {
        "created_at": "2020-08-20T16:47:11.753Z",
        "guid": "448df622-7b06-4fab-a92f-ad2a978f1a2a",
        "url": "https://us-south.ml.cloud.ibm.com/v4/runtimes/448df622-7b06-4fab-a92f-ad2a978f1a2a"
    }
}


#### Criando uma nova definição de Pipeline personalizada no WML

Finalmente iremos criar uma definição (metadados) para a nossa Pipeline ser hospedada no WML.

Definimos como parâmetros um nome para o artefato e o ID do runtime criado anteriormente.

In [24]:
model_meta = {
    clientWML.repository.ModelMetaNames.NAME: 'desafio-2-mbtc2020-pipeline-2',
    clientWML.repository.ModelMetaNames.DESCRIPTION: "my pipeline for submission",
    clientWML.repository.ModelMetaNames.RUNTIME_UID: custom_runtime_uid
}

Em seguida chamamos o método para armazenar a nova definição:

In [25]:
# Função para armazenar uma definição de Pipeline no WML
stored_model_details = clientWML.repository.store_model(
    model=my_pipeline,  # `my_pipeline` é a variável criada anteriormente e contém nossa Pipeline já treinada :)
    meta_props=model_meta,  # Metadados definidos na célula anterior
    training_data=None  # Não altere esse parâmetro
)

print("\n Lista de artefatos armazenados no WML:")
clientWML.repository.list()

# Detalhes do modelo hospedado no Watson Machine Learning
print("\n Metadados do modelo armazenado:")
print(json.dumps(stored_model_details, indent=4))


 Lista de artefatos armazenados no WML:
------------------------------------  -----------------------------  ------------------------  -----------------  --------------
GUID                                  NAME                           CREATED                   FRAMEWORK          TYPE
e6419c3a-fb08-4db6-b6ba-12c868f2a3ca  desafio-2-mbtc2020-pipeline-2  2020-08-20T16:47:24.711Z  scikit-learn-0.19  model
d17740e4-6f74-48ec-b4a8-192d56cb431e  desafio-2-mbtc2020-pipeline-1  2020-08-20T16:23:16.430Z  scikit-learn-0.20  model
dfde0b21-31fa-4258-8f8a-e06cf2528772  my_custom_sklearn_transform_2  2020-08-20T16:47:01.758Z  -                  python library
2c13ace2-6793-40a5-88ac-6c3e058c711a  my_custom_sklearn_transform_1  2020-08-20T16:22:59.435Z  -                  python library
448df622-7b06-4fab-a92f-ad2a978f1a2a  my_custom_wml_runtime_2        2020-08-20T16:47:11.753Z  -                  python runtime
89f62cc9-17b3-4a5a-9c2f-68817be13f90  my_custom_wml_runtime_1        2020-08-20T16:2

#### Realizando o deployment do seu modelo para consumo imediato por outras aplicações

In [26]:
# O deployment do modelo é finalmente realizado por meio do método ``deployments.create()``

model_deployment_details = clientWML.deployments.create(
    artifact_uid=stored_model_details["metadata"]["guid"],  # Não altere esse parâmetro
    name="desafio-2-mbtc2020-deployment-2",
    description="Solução do desafio 2 - MBTC",
    asynchronous=False,  # Não altere esse parâmetro
    deployment_type='online',  # Não altere esse parâmetro
    deployment_format='Core ML',  # Não altere esse parâmetro
    meta_props=model_meta  # Não altere esse parâmetro
)



#######################################################################################

Synchronous deployment creation for uid: 'e6419c3a-fb08-4db6-b6ba-12c868f2a3ca' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS.........................................
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='22e8c150-3a5e-4b31-9ea8-a6c858f5c2e0'
------------------------------------------------------------------------------------------------




#### Testando um modelo hospedado no Watson Machine Learning

In [27]:
# Recuperando a URL endpoint do modelo hospedado na célula anterior

model_endpoint_url = clientWML.deployments.get_scoring_url(model_deployment_details)
print("A URL de chamada da sua API é: {}".format(model_endpoint_url))

A URL de chamada da sua API é: https://us-south.ml.cloud.ibm.com/v3/wml_instances/e9155f37-76f9-4b5d-a011-7eaf4c0b10f6/deployments/22e8c150-3a5e-4b31-9ea8-a6c858f5c2e0/online


In [28]:
# Detalhes do deployment realizado

deployment_details = clientWML.deployments.get_details(
    deployment_uid=model_deployment_details["metadata"]["guid"]  # esse é o ID do seu deployment!
)

print("Metadados do deployment realizado: \n")
print(json.dumps(deployment_details, indent=4))

Metadados do deployment realizado: 

{
    "metadata": {
        "guid": "22e8c150-3a5e-4b31-9ea8-a6c858f5c2e0",
        "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/e9155f37-76f9-4b5d-a011-7eaf4c0b10f6/deployments/22e8c150-3a5e-4b31-9ea8-a6c858f5c2e0",
        "created_at": "2020-08-20T16:47:43.428Z",
        "modified_at": "2020-08-20T16:47:44.253Z"
    },
    "entity": {
        "runtime_environment": "python-3.6",
        "name": "desafio-2-mbtc2020-deployment-2",
        "scoring_url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/e9155f37-76f9-4b5d-a011-7eaf4c0b10f6/deployments/22e8c150-3a5e-4b31-9ea8-a6c858f5c2e0/online",
        "deployable_asset": {
            "name": "desafio-2-mbtc2020-pipeline-2",
            "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/e9155f37-76f9-4b5d-a011-7eaf4c0b10f6/published_models/e6419c3a-fb08-4db6-b6ba-12c868f2a3ca",
            "guid": "e6419c3a-fb08-4db6-b6ba-12c868f2a3ca",
            "description": "my pipel

In [29]:
scoring_payload = {
    'fields': [
        "MATRICULA", "NOME", 'REPROVACOES_DE', 'REPROVACOES_EM', "REPROVACOES_MF", "REPROVACOES_GO",
        "NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO",
        "INGLES", "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
    ],
    'values': [
        [
            513949,"Marli Quésia de Oliveira",1,1,1,1,4.3,4.0,3.1,4.9,0,3,4,3,
        ]
    ]
}

print("\n Payload de dados a ser classificada:")
print(json.dumps(scoring_payload, indent=4))


 Payload de dados a ser classificada:
{
    "fields": [
        "MATRICULA",
        "NOME",
        "REPROVACOES_DE",
        "REPROVACOES_EM",
        "REPROVACOES_MF",
        "REPROVACOES_GO",
        "NOTA_DE",
        "NOTA_EM",
        "NOTA_MF",
        "NOTA_GO",
        "INGLES",
        "H_AULA_PRES",
        "TAREFAS_ONLINE",
        "FALTAS"
    ],
    "values": [
        [
            513949,
            "Marli Qu\u00e9sia de Oliveira",
            1,
            1,
            1,
            1,
            4.3,
            4.0,
            3.1,
            4.9,
            0,
            3,
            4,
            3
        ]
    ]
}


In [30]:
result = clientWML.deployments.score(
    model_endpoint_url,
    scoring_payload
)

print("\n Resultados:")
print(json.dumps(result, indent=4))


 Resultados:
{
    "fields": [
        "prediction",
        "probability"
    ],
    "values": [
        [
            "DIFICULDADE",
            [
                0.9469998478889465,
                0.036890290677547455,
                0.005289118271321058,
                0.005200723651796579,
                0.005620004143565893
            ]
        ]
    ]
}


<hr>

## Parabéns! 

Se tudo foi executado sem erros, você já tem um classificador baseado em machine learning encapsulado como uma API REST!

Para testar a sua solução integrada com um assistente virtual e realizar a submissão, acesse a página:

https://uninassau.maratona.dev

Você irá precisar da endpoint url do seu modelo e das credenciais do WML :)